
<h1><center> Προσπαρασκευή 1ης εργαστηριακής άσκησης:</center></h1>
<h2><center> Μέρος πρώτο: Ορθογράφος</center><h2>

### ΕΚΤΕΛΕΣΗ

### Βήμα 1: Κατασκευή corpus

__α)__ Κατεβάζουμε λοιπόν από το project Gutenberg το βιβλίο __War of the Worlds__ σε plain txt μορφή και το αποθηκεύουμε με το όνομα __War.txt__ (με το ! στην αρχή μιάς εντολής δηλώνουμε ότι ακολουθεί ένα shell command). 

In [1]:
! wget  -c http://www.gutenberg.org/files/36/36-0.txt -O War.txt

--2018-11-19 19:20:28--  http://www.gutenberg.org/files/36/36-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 366813 (358K) [text/plain]
Saving to: ‘War.txt’

War.txt             100%[===================>] 358,22K   462KB/s    in 0,8s    

2018-11-19 19:20:29 (462 KB/s) - ‘War.txt’ saved [366813/366813]



__β)__ Προφανώς μπορούμε να ενώσουμε περισσότερα βιβλία για την κατασκευή ενός μεγαλύτερου corpus. Τα πλεονεκτήματα αυτής της τεχνικής (εκτός από την αύξηση του μεγέθους των δεδομένων) είναι:
1. Τα δεδομένα μας θα προέρχονται από πολλές διαφορετικές πηγές με αποτέλεσμα το γλωσσικό μοντέλο μας να γενικεύσει περισσότερο και να μην κάνει overfit στα δεδομένα μας.
2. Όταν έχουμε ένα μεγάλο corpus τα στατιστικά στοιχεία που θα υπολογίσουμε πάνω σε αυτό θα είναι ακριβέστερα και ακόμα θα είμαστε σε θέση να κρίνουμε ποιό στατιστικό στοιχείο είναι σημαντικότερο για το μοντέλο μας.

### Βήμα 2: Προεπεξεργασία corpus

Έχουμε λοιπόν το __corpus__ που θα χρησιμοποιήσουμε στον ίδιο φάκελο με το notebook μας. Θα ορίσουμε τώρα ορισμένες χρήσιμες συναρτήσεις για την προεπεξεργασία του.

__α)__ Η συνάρτηση __identity_preprocess__ διαβάζει ένα string και απλά γυρνάει τον εαυτό του.

In [2]:
def identity_preprocess(string_var):
    return string_var

__β)__ Η συνάρτηση __read_path__ δέχεται σαν όρισμα το path του αρχείου που θέλουμε να διαβάσουμε και μία συνάρτηση preprocess και διαβάζει το αρχείο γραμμή προς γραμμή σε μία λίστα, καλώντας την preprocess σε κάθε γραμμή. Χρησιμοποιούμε σαν default όρισμα για την preprocess την identity_preprocess που ορίστηκε παραπάνω.

In [3]:
def read_path(file_path, preprocess = identity_preprocess):
    #initilize the list of processed lines
    processed_lines = []
    #open file to read mode
    with open(file_path, "r") as f:
        for line in f:
            #omit spaces
            if not line.isspace():
                processed_lines.extend(preprocess(line))
    return processed_lines

__γ)__ Η συνάρτηση __tokenize__ δέχεται σαν όρισμα ένα string s και: 
    1. καλεί την strip() και lower() πάνω στο s, 
    2. αφαιρεί όλα τα σημεία στίξης / σύμβολα / αριθμούς, αφήνωντας μόνο αλφαριθμητικούς χαρακτήρες, 
    3. αντικαθιστά τα newlines με κένα, 
    4. κάνει split() τις λέξεις στα κενά.

In [4]:
import string 

def tokenize(s):
    #Remove possible spaces from the start or the end of the string and
    #turn all letters lowercase.
    s = s.strip().lower()
    #Remove all punctuations, symbols and numbers from the string leaving
    #only lowercase alphabetical letters.
    s = "".join((char for char in s if char not in string.punctuation and not char.isdigit()))
    #Replace new line characters with spaces
    s = s.replace('\n',' ')
    #Split the string in every space resulting in a list of tokens
    res = s.split(" ")
    return res

__δ)__ Μπορούμε αντί να κατασκευάσουμε τον δικό μας tokenizer να χρησιμοποιήσουμε έναν έτοιμο από την βιβλιοθήκη nltk που διαθέτει πόλλους διαφορετικούς tokenizer. Παρακάτω ορίζουμε έναν από αυτούς τον οποίο θα συγκρίνουμε αργότερα με τον δικό μας.

In [5]:
from nltk.tokenize import WordPunctTokenizer

def nltk_tokenizer(s):
    words = WordPunctTokenizer().tokenize(text = s)
    words = [word.lower() for word in words if word.isalpha() and not word.isdigit()]
    return words

Στη συνέχεια μπορούμε να συγκρίνουμε τους δύο tokenizers δίνοντας τους ως είσοδο ένα string με διάφορα είδη χαρακτήρων, συμβόλων και αριθμών. Παρατηρούμε ότι και οι δύο ανταποκρίνονται πολύ καλά δίνοντάς μας το επιθυμητό αποτέλεσμα, απλά ο δικός μας αποθηκεύει και κάποια κενά ως token το οποίο μπορεί να διορθωθεί κατά την δημιουργία του λεξικού.

In [6]:
test_string = "hi my !@( name 32 is ... Panos"
print("My tokenizer :" + " ", end="")
print(tokenize(test_string))
print("Nltk tokenizer :" + " ", end="")
print(nltk_tokenizer(test_string))

My tokenizer : ['hi', 'my', '', 'name', '', 'is', '', 'panos']
Nltk tokenizer : ['hi', 'my', 'name', 'is', 'panos']


### Βήμα 3: Κατασκευή λεξικού και αλφαβήτου

__α)__ Δημιουργούμε μία λίστα η οποία θα περιέχει όλα τα μοναδικά tokens που βρίσκονται στο corpus που θα χρησιμοποιήσουμε, σύμφωνα με τον tokenizer του Βήματος 2. Ουσιαστικά θα καλέσουμε την συνάρτηση read_path με όρισμα τον tokenizer που θέλουμε να εφαρμοστεί σε κάθε γραμμή του αρχείου και στη συνέχεια θα πάρουμε μόνο τις μοναδικές λέξεις για να σχηματίσουμε το λεξικό μας.

In [7]:
def get_tokens(file_path):
    tokens = read_path(file_path, tokenize)
    distinct_tokens = list(dict.fromkeys(tokens))
    return distinct_tokens

In [8]:
tokens_dict = get_tokens("War.txt")

__β)__ Δημιουργούμε μία λίστα που περιέχει το αλφάβητο του corpus.

In [9]:
def get_alphabet(tokens):
    alphabet = []
    for token in tokens:
        alphabet.extend(list(token))
    alphabet = list(dict.fromkeys(alphabet))
    return alphabet

In [10]:
alphabet = get_alphabet(tokens_dict)

### Βήμα 4: Δημιουργία συμβόλων εισόδου-εξόδου

Η συνάρτηση __alphabet _to_int__ δέχεται ως όρισμα το αλφάβητο που έχουμε ορίσει από το corpus, αντιστοιχίζει κάθε χαρακτήρα σε ένα αύξοντα ακέραιο index και γράφει το αποτέλεσμα σε ένα αρχείο chars.syms με αυτή τη μορφή http://www.openfst.org/twiki/pub/FST/FstExamples/ascii.syms. Σε όλα τα επόμενα βήματα θα χρησιμοποιήσουμε το αρχείο chars.syms για τα σύμβολα εισόδου και εξόδου των FSTs.

In [11]:
def alphabet_to_int(alphabet):
    # open file
    f = open("chars.syms", "w")
    # match epsilon to 0
    f.write("EPS" + 7*" " + str(0) + '\n')
    num = 21
    for character in alphabet:
        # match every other character to an increasing index
        f.write(character + 7*" " + str(num) + '\n')
        num += 1
    f.close()

In [12]:
alphabet_to_int(alphabet)

### Βήμα 5: Κατασκευή μετατροπέων FST

__α)__ Για να κατασκευάσουμε τον μετατροπέα σε αυτό το βήμα αλλά και τον αποδοχέα στο επόμενο θα χρειαστούμε μία συνάρτηση format_arc η οποία θα διαμορφώνει μία γραμμή από το αρχείο περιγραφής του κάθε FST. Συγκεκριμένα δέχεται ως όρισμα τα __src__, __dest__, __ilabel__, __olabel__ και το __weight__ (με default τιμή το 0) και τα επιστρέφει στην κατάλληλη μορφή όπως αναφέρεται και εδώ http://www.openfst.org/twiki/bin/view/FST/FstQuickTour#CreatingFsts/.

In [13]:
def format_arc(src, dest, ilabel, olabel, weight=0):
    return (str(src) + " " + str(dest) + " " + str(ilabel) + " " + str(olabel) + " " + str(weight))

Τώρα λοιπόν θα διαμορφώσουμε το αρχείο περιγραφής του μετατροπέα μας, ο οποίος θα έχει μία κατάσταση και θα υλοποιεί την απόσταση Levenshtein αντιστοιχίζοντας:
1. Κάθε χαρακτήρα στον εαυτό του με βάρος 0 (no edit).
2. Κάθε χαρακτήρα στο EPS με βάρος 1 (deletion).
3. Το EPS σε κάθε χαρακτήρα με βάρος 1 (insertion).
4. Κάθε χαρακτήρα σε κάθε άλλο χαρακτήρα με βάρος 1 (substitution).

Η περιγραφή του μετατροπέα αποθηκεύεται στο αρχείο __transducer.fst__.

In [14]:
# get alphabet of the corpus
alphabet = get_alphabet(get_tokens("War.txt"))
# open file to write mode
f = open("transducer.fst", "w")
for letter in alphabet:
    # no edit
    f.write(format_arc(0, 0, letter, letter) + "\n")
    # deletion
    f.write(format_arc(0, 0, letter, "EPS", 1) + "\n")
    # insertion
    f.write(format_arc(0, 0, "EPS", letter, 1) + "\n")
for i in range(len(alphabet)):
    for j in range(len(alphabet)):
        if i != j:
            # substitution
            f.write(format_arc(0, 0, alphabet[i], alphabet[j], 1) + "\n")

# make initial state also final state
f.write("0")
# close file
f.close()

Τρέχοντας λοιπόν το παρακάτω shell command κάνουμε compile τον μετατροπέα μας. Το binary αρχείο που προκύπτει με όνομα __transducer.bin.fst__ είναι αυτό που θα χρησιμοποιήσουμε στις επόμενες λειτουργίες.

In [15]:
! ./compile_transducer.sh

Είναι προφανές ότι ο μετατροπέας μας αυτή την στιγμή σε μία λέξη εισόδου αν πάρουμε το shortest path απλά θα επιστρέψει την ίδια την λέξη αυτούσια χωρίς καμμία αλλαγή. Και αυτό γιατί για κάθε σύμβολο του αλφαβήτου έχουμε μετάβαση στον εαυτό του με κόστος 0. Ο μετατροπέας μας θα αποκτήσει νόημα όταν τον συνθέσουμε με τον αποδοχέα του επόμενου ερωτήματος.

__β)__ Αυτός είναι ένας αρκετά αφελής τρόπος για τον υπολογισμό των βαρών για κάθε edit. Αν τώρα είχαμε στη διάθεση μας ό,τι δεδομένα θέλουμε αυτό που θα κάναμε είναι ότι θα υπολογίζαμε τα βάρη με βάση το πόσο συχνά γίνεται αυτό το λάθος. Πιο συγκεκριμένα, θα υπολογίζαμε για κάθε σύμβολο του αλφαβήτου την πιθανότητα κάποιος να το διαγράψει, να το προσθέσει ή να το αντικαταστήσει με κάποιο άλλο. Στη συνέχεια, θα μετατρέπαμε αυτές τις πιθανότητες σε κόστη παίρνοντας τον αρνητικό λογάριθμο και θα είχαμε τα τελικά βάρη μας για κάθε σύμβολο στο deletion και το insertion και για κάθε δυάδα συμβόλων στο substitution.

### Βήμα 6: Κατασκευή αποδοχέα λεξικού

__α)__ Σε αυτό το βήμα θέλουμε να κατασκευάσουμε έναν αποδοχέα με μία αρχική κατάσταση που αποδέχεται κάθε κατάσταση του λεξικού μας. Τα βάρη όλων των ακμών θα είναι 0. Αντίστοιχα με πριν γράφουμε την περιγραφή του αποδοχέα σε ένα αρχείο __acceptor.fst__.

In [16]:
# get tokens of the corpus (our acceptor should accept only these words)
tokens = get_tokens("War.txt")
# open file to write mode
f = open("acceptor.fst", "w")
s = 1
for token in tokens:
    letters = list(token)
    for i in range(0, len(letters)):
        if i == 0:
            # For each token make state 1 its first state
            f.write(format_arc(1, s+1, letters[i], letters[i]) + "\n")
        else:
            f.write(format_arc(s, s+1, letters[i], letters[i]) + "\n")
        s += 1
        if i == len(letters) - 1:
            # When reaching the end of a token go to final state 0 though an ε-transition
            f.write(format_arc(s, 0, "EPS", "EPS") + "\n")
# make state 0 final state
f.write("0")
# close the file
f.close()

 Στη συνέχεια το fst μας γίνεται compile στο αρχείο __acceptor.bin.fst__.

In [17]:
! ./compile_acceptor.sh

__β)__ Στη συνέχεια καλούμε τις συναρτήσεις fstrmepsilon, fstdeterminize και fstminimize για να βελτιστοποιήσουμε το μοντέλο μας. Παρακάτω αναφέρεται η λειτουργία της κάθεμίας:
1. __fstrmepsilon__: Μετατρέπει το FST σε ένα ισοδύναμο που να μην περιέχει ε-μεταβάσεις
2. __fstdeterminize__: Μετατρέπει το FST σε ένα ισοδύναμο ντετερμινιστικό. Το τελικό FST, δηλαδή, θά έχει για κάθε κατάσταση και για κάθε σύμβολο εισόδου μία και μόνο μετάβαση.
2. __fstminimize__: Ελαχιστοποιεί το FST ώστε να έχει τον ελάχιστο αριθμό καταστάσεων.

In [18]:
! fstrmepsilon acceptor.bin.fst acceptor.bin.fst

In [19]:
! fstdeterminize acceptor.bin.fst acceptor.bin.fst

In [20]:
! fstminimize acceptor.bin.fst acceptor.bin.fst

### Βήμα 7: Κατασκευή ορθογράφου

Στο βήμα αυτό θα συνθέσουμε τον Levenshtein transducer με τον αποδοχέα του ερωτήματος 6α παράγοντας τον min edit distance spell checker. Αρχικά θα ταξινομήσουμε τις εξόδους του transducer και τις εισόδους του αποδοχέα με την συνάρτηση __fstarcsort__.

In [21]:
! fstarcsort --sort_type=olabel transducer.bin.fst transducer_sorted.fst

In [22]:
! fstarcsort --sort_type=ilabel acceptor.bin.fst acceptor_sorted.fst

Στη συνέχεια συνθέτουμε τον transducer με τον αποδοχέα με την συνάρτηση fstcompose αποθηκεύοντας τον min edit distance spell checker στο αρχείο __spell_checker.fst__.

In [23]:
! fstcompose transducer_sorted.fst acceptor_sorted.fst spell_checker.fst

Η συμπεριφορά του μετατροπέα που έχουμε υλοποιήσει διαφέρει ανάλογα με τα βάρη που θα βάλουμε σε κάθε edit. Συγκεκριμένα:
1. Στην περίπτωση που τα edits είναι ισοβαρή έχουμε ένα αφελή transducer που απλά διορθώνει τις λέξεις χωρίς να λαμβάνει υπόψιν του κάποια γλωσσική πληροφορία, με μόνο κριτήριο να κάνει τις ελάχιστες δυνατές μετατροπές στην λέξη εισόδου.
2. Στην περίπτωση τώρα που τα edits δεν είναι ισοβαρή μπορούμε να υλοποιήσουμε έναν πιο ισχυρό trandsucer. Συγκεκριμένα, μπορούμε να ορίσουμε με βάση κάποια γνωστά μοντέλα την πιθανότητα ένα σύμβολο να λείπει, να έχει προστεθεί λανθασμένα ή να έχει αντικατασταθεί από κάποιο άλλο σύμβολο και να μετατρέψουμε αυτές τις πιθανότητες σε κόστη τα οποία θα μεταφραστούν στη συνέχεια ως βάρη στα αντίστοιχα edits. Έτσι, η λέξη θα διορθώνεται όχι με βάση τις ελάχιστες δυνατές μετατροπές αλλά με το ποιά λέξη είναι πιο πίθανο να ήθελε να γράψει αυτός που την έγραψε λάθος.

__β)__  Στην περίπτωση, λοιπόν, που η είσοδος στον min edit spell checker μας είναι η λέξη __cit__ θα προσπαθήσει με τον ελάχιστο αριθμό που τις τρεις λειτουργίες που του επιτρέπονται (insert, delete, substitute) να την μετατρέψει σε μία λέξη που υπάρχει στο λεξικό μας. Επειδή, βέβαια, το λεξικό μας προέρχεται απλά από ένα βιβλίο είναι λογικό πολλές δημοφιλής λέξεις να λείπουν. Στην περίπτωση μας έχουμε τις εξής πιθανές προβλέψεις:
1. Με μία αλλαγή μόνο:
  * __cat__
  * __sit__
  * __fit__
  * __cut__
  * __cite__
2. Σε περίπτωση τώρα που καμία από τις παραπάνω λέξεις δεν βρίσκεται στο λεξικό μας ο ορθογράφος θα κοιτάξει στις λέξεις που προκύπτουν από δύο αλλαγές στις λέξεις κ.ο.κ.

### Βήμα 8: Αξιολόγηση ορθογράφου

__α)__ Για να κάνουμε το evaluation του ορθογράφου κατεβάζουμε το παρακάτω σύνολο δεδομένων:

In [24]:
! wget https://raw.githubusercontent.com/georgepar/python-lab/master/spell_checker_test_set

--2018-11-19 19:20:36--  https://raw.githubusercontent.com/georgepar/python-lab/master/spell_checker_test_set
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3780 (3,7K) [text/plain]
Saving to: ‘spell_checker_test_set’

spell_checker_test_ 100%[===================>]   3,69K  --.-KB/s    in 0s      

2018-11-19 19:20:39 (10,2 MB/s) - ‘spell_checker_test_set’ saved [3780/3780]



__β)__ Από το παραπάνω test set λοιπόν θα επιλεξουμε 20 λέξεις στην τύχη και με βάση τον αλγόριθμο των ελάχιστων μονοπατιών στο γράφο του μετατροπέα του βήματος 7 θα την διορθώσουμε και θα εκτιμήσουμε την διόρθωση.

Δημιουργούμε αρχικά μία συνάρτηση __predict__ η οποία δέχεται μία λέξη που πρέπει να διορθωθεί και γράφει σε ένα αρχείο __pred_word.fst__ την περιγραφή ενός FST το οποίο αποδέχεται την συγκεκριμένη λέξη. Το FST αυτό θα το κάνουμε στη συνέχεια compose με τον ορθογράφο μας για να πάρουμε το τελικό αποτέλεσμα.

In [25]:
def predict(word):
    s= 1
    letters = list(word)
    # open file to write mode
    f = open("pred_word.fst", "w")
    for i in range(0, len(letters)):
        # for each letter of the word make a transition with zero weight
        f.write(format_arc(s, s+1, letters[i], letters[i], 0) + '\n')
        s += 1
        if i == len(letters) - 1:
            # when reaching the end the word make a ε-transition to the final state 0 
            f.write(format_arc(s, 0, "EPS",  "EPS", 0) + '\n')
    # final state
    f.write("0")
    # close the file
    f.close()

Στη συνέχεια, επιλέγουμε από το test set 20 τυχαίες λέξεις. Συγκεκριμένα, επιλέγουμε 20 τυχαίες σειρές και από κάθε σειρά επιλέγουμε τυχαία μία λανθασμένη λέξη. Βάζουμε ένα συγκεκριμένο seed έτσι ώστε να προκύπτουν οι ίδιες λέξεις κάθε φορά και να γίνει η εκτίμηση αργότερα.

In [26]:
import random
random.seed(1)
test_words = []
for _ in range(20):
    random_lines = random.choice(open('spell_checker_test_set').readlines())
    test_words.append(random.choice(random_lines.strip('\n').split()[1:]))

Τέλος, για κάθε λέξη καλούμε την συνάρτηση predict και στη συνέχεια τρέχουμε στο bash (με το σύμβολο !) το αρχείο predict.sh δίνοντας ως όρισμα τον ορθογράφο μας.

In [27]:
for word in test_words:
    print(word + ":" + " ",end='')
    predict(word)
    ! ./predict.sh spell_checker.fst
    print('\n')

experances: experiences

chosing: chasing

extreamly: extremely

cirtain: certain

completly: completely

diffrent: different

aranging: hanging

contentid: contents

awfall: fall

defenition: desertion

compair: complain

localy: local

transfred: transfixed

experance: experience

adress: dress

vistors: visitors

rememmer: remember

remeber: remember

failes: failed

juise: noise



##### Σχολιασμός αποτελεσμάτων: 
Παρατηρούμε ότι ο ορθογράφος λειτουργεί με μικρή ακρίβεια. Υπάρχουν συγκεκριμένα λάθη σε κάποιες λέξεις που οφείλονται στο γεγονός ότι τα βάρη στον μετατροπέα είναι όλα 1 και ότι το corpus μας αποτελείται μόνο από ένα βιβλίο.
1. __chosing__ το οποίο διορθώνεται σε chasing αλλά θα μπορούσε να διορθωθεί σε choosing το οποίο έχει ίδιο αριθμό αλλαγών με το chasing αλλά θα μπορύσαμε να πούμε ότι όταν δύο γράμματα είναι συνεχόμενα είναι πολύ πιθανό να παραλειφθεί το ένα από τα δύο. Άρα το λάθος αυτό οφείλεται στά ίδια βάρη.
2. __aranging__, __awfall__, __defenition__, __compair__, __transfred__, __adress__, __juise__ που θα μπορούσαν να γίνουν με λιγότερα edits σε arranging,awful, definition, compare, transfered, address, juice αντίστοιχα. Τα λάθη αυτά οφείλονται στο ότι οι συγκεκριμένες λέξεις δεν υπήρχας στο corpus μας.

### Βήμα 9: Εξαγωγή αναπαραστάσεων word2vec

__α)__ Αρχικά θα διαβάσουμε το κείμενο σε μία λίστα από tokenized προτάσεις χρησιμοποιώντας τον κώδικα του βήματος 2γ. Για να χωρίσουμε το corpus σε προτάσεις χρησιμοποιούμε την βιβλιοθήκη __nltk__.

In [28]:
import nltk

# We split the corpus in a list of tokenized sentences.
file_path = "War.txt"
tokenized_sentences = []
with open(file_path, "r") as f:
    text = f.read()
    sentences = nltk.sent_tokenize(text)
    tokenized_sentences = [tokenize(sentence) for sentence in sentences]

__β)__ Θα χρησιμοποιήσουμε την κλάση Word2Vec του gensim για να εκπαιδεύσουμε 100-διάστατα word2vec embeddings με βάση τις προτάσεις του βήματος 9α. Θα χρησιμοποιήσουμε window = 5 και 1000 εποχές.

In [29]:
from gensim.models import Word2Vec

# Initialize word2vec. Context is taken as the 2 previous and 2 next words
model = Word2Vec(tokenized_sentences, window=5, size=100, workers=4)
# Train the model for 1000 epochs
model.train(tokenized_sentences, total_examples=len(sentences), epochs=1000)

(35586089, 63502000)

In [30]:
# get ordered vocabulary list
voc = model.wv.index2word
# get vector size
dim = model.vector_size

In [31]:
import numpy as np

# Convert to numpy 2d array (n_vocab x vector_size)
def to_embeddings_Matrix(model):  
    embedding_matrix = np.zeros((len(model.wv.vocab), model.vector_size))
    for i in range(len(model.wv.vocab)):
        embedding_matrix[i] = model.wv[model.wv.index2word[i]]
    return embedding_matrix, model.wv.index2word

__γ)__ Θα επιλέξουμε 10 τυχαίες λέξεις από το λεξικό μας και θα βρούμε τις σημασιολογικά κοντινότερές τους.

In [37]:
import random

selected_words = random.sample(voc, 10)
for word in selected_words:
    # get most similar words
    sim = model.wv.most_similar(word, topn=5)
    print('"' + word + '"' + " is similar with the following words:")
    for s in sim:
        print('"' + s[0] + '"' + " with similarity " + str(s[1]))
    print()

"happening" is similar with the following words:
"doing" with similarity 0.3493283987045288
"traffic" with similarity 0.32823145389556885
"met" with similarity 0.2843232750892639
"vi" with similarity 0.28040194511413574
"heat" with similarity 0.24910134077072144

"also" is similar with the following words:
"lay" with similarity 0.3623203635215759
"reach" with similarity 0.3022865056991577
"artilleryman" with similarity 0.2998621463775635
"believe" with similarity 0.2931804060935974
"curious" with similarity 0.28869128227233887

"kingston" is similar with the following words:
"richmond" with similarity 0.35499098896980286
"southwestern" with similarity 0.3271881341934204
"wimbledon" with similarity 0.3082829713821411
"regent’s" with similarity 0.28964874148368835
"surrey" with similarity 0.2789665460586548

"ideas" is similar with the following words:
"evening" with similarity 0.38338983058929443
"left" with similarity 0.3477540910243988
"fresh" with similarity 0.32628852128982544
"terr

/home/panos/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Θα επιλέξουμε τώρα μεγαλύτερο παράθυρο κρατώντας ίδιο τον αριθμό των εποχών.

In [33]:
# Initialize word2vec. Context is taken as the 2 previous and 2 next words
model = Word2Vec(tokenized_sentences, window=20, size=100, workers=4)
# Train the model for 1000 epochs
model.train(tokenized_sentences, total_examples=len(sentences), epochs=1000)

(35588166, 63502000)

In [34]:
# get ordered vocabulary list
voc = model.wv.index2word
# get vector size
dim = model.vector_size

import random

selected_words = random.sample(voc, 10)
for word in selected_words:
    # get most similar words
    sim = model.wv.most_similar(word, topn=5)
    print('"' + word + '"' + " is similar with the following words:")
    for s in sim:
        print('"' + s[0] + '"' + " with similarity " + str(s[1]))
    print()

"thirst" is similar with the following words:
"disturbed" with similarity 0.35343295335769653
"immediate" with similarity 0.35186639428138733
"mental" with similarity 0.3263455331325531
"raised" with similarity 0.3096556067466736
"carts" with similarity 0.293457955121994

"parted" is similar with the following words:
"driven" with similarity 0.3943368196487427
"wood" with similarity 0.3792082667350769
"being" with similarity 0.34603962302207947
"tripod" with similarity 0.3304838538169861
"george’s" with similarity 0.3066193461418152

"waiting" is similar with the following words:
"electric" with similarity 0.3446633815765381
"go" with similarity 0.3046344816684723
"pine" with similarity 0.27193909883499146
"editions" with similarity 0.27186113595962524
"east" with similarity 0.27152714133262634

"eight" is similar with the following words:
"distribute" with similarity 0.36154472827911377
"meat" with similarity 0.3328061103820801
"using" with similarity 0.3102995753288269
"hedge" with s

/home/panos/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Αντίθετα τώρα θα αυξήσουμε τον αριθμό των εποχών κρατώντας το μέγεθος του παραθύρου σταθερό.

In [35]:
# Initialize word2vec. Context is taken as the 2 previous and 2 next words
model = Word2Vec(tokenized_sentences, window=5, size=100, workers=4)
# Train the model for 1000 epochs
model.train(tokenized_sentences, total_examples=len(sentences), epochs=3000)

(106767209, 190506000)

In [36]:
# get ordered vocabulary list
voc = model.wv.index2word
# get vector size
dim = model.vector_size

import random

selected_words = random.sample(voc, 10)
for word in selected_words:
    # get most similar words
    sim = model.wv.most_similar(word, topn=5)
    print('"' + word + '"' + " is similar with the following words:")
    for s in sim:
        print('"' + s[0] + '"' + " with similarity " + str(s[1]))
    print()

"sufficiently" is similar with the following words:
"sign" with similarity 0.309844434261322
"kill" with similarity 0.30193138122558594
"dull" with similarity 0.29572296142578125
"take" with similarity 0.27504679560661316
"get" with similarity 0.27170515060424805

"soldier" is similar with the following words:
"touched" with similarity 0.30009931325912476
"felt" with similarity 0.29300981760025024
"digging" with similarity 0.27054011821746826
"either" with similarity 0.25615382194519043
"wwwgutenbergorg" with similarity 0.248465895652771

"best" is similar with the following words:
"latter" with similarity 0.3304646611213684
"description" with similarity 0.30726027488708496
"former" with similarity 0.3039565682411194
"taking" with similarity 0.2877877354621887
"excited" with similarity 0.2753252685070038

"hurry" is similar with the following words:
"kind" with similarity 0.417825311422348
"doubt" with similarity 0.3668208420276642
"bread" with similarity 0.3250107169151306
"world" wit

/home/panos/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


#### Σχολιασμός Αποτελεσμάτων

Τα αποτελέσματα δεν είναι τόσο ποιοτικά όσο περιμέναμε. Παρατηρούμε ότι σε κάποιες περιπτώσεις πετυχαίνει 1-2 λέξεις που όντως είναι σημασιολογικά κοντά με την επιλεγόμενη λέξη. Ακόμα και όταν αυξήσαμε το μέγεθος του παραθύρου ή τον αριθμό των εποχών η κατάσταση παρέμεινε η ίδια.Κατά την γνώμη μου αυτό συμβαίνει διότι έχουμε ένα πολύ μικρό corpus που δεν βοηθάει το μοντέλο ώστε να καταλάβει ποιες λέξεις είναι πραγματικά κοντά σημασιολογικά.